In [1]:
import torch
from torch_geometric.datasets import TUDataset
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch.utils.data import random_split
from experiments.classifier import GraphClassifierTimeConv
from experiments.train import train, test, cross_validate
from torch.optim import Adam
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the dataset
dataset_name = "ENZYMES"
path = "./data/tmp/ENZYMES"
original_dataset = TUDataset(root=path, name=dataset_name, use_node_attr=True)

# Function to add equal edge weights to a graph
def add_edge_weights(data):
    edge_index = data.edge_index
    edge_weight = torch.ones(edge_index.shape[1], dtype=torch.float)
    return Data(edge_index=edge_index, x=data.x, y=data.y, edge_weight=edge_weight)

# Create a new dataset with added edge weights
dataset = [add_edge_weights(data) for data in original_dataset]
n_input = dataset[0].x.size(1)

# Split the dataset into train and test sets
# train_size = int(0.8 * len(dataset))  # 80% of the dataset for training
# test_size = len(dataset) - train_size

# train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create DataLoaders for train and test sets
# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [2]:
ts = torch.linspace(0, 1, 20, dtype=torch.float, device=device)
# Create your model, optimizer, and criterion
model = GraphClassifierTimeConv(pde='heat', ts=ts, n_input=n_input, n_hidden=16, n_output=6, n_time_conv=5, n_time_pool=5, device=device).to(device)
optimizer = Adam(model.parameters(), lr=1e-3)
# criterion = CrossEntropyLoss()
criterion = torch.nn.NLLLoss()

# CV
results_heat = cross_validate(
    model, 
    dataset, 
    n_splits=5, 
    batch_size=1024, 
    num_epochs=1000, 
    lr=1e-3, 
    weight_decay=0., 
    device=device
)

In [3]:
ts = torch.linspace(0., torch.pi, 20, dtype=torch.float, device=device)
# Create your model, optimizer, and criterion
model = GraphClassifierTimeConv(pde='wave', ts=ts, n_input=n_input, n_hidden=16, n_output=6, n_time_conv=5, n_time_pool=5, device=device).to(device)
optimizer = Adam(model.parameters(), lr=1e-3)
# criterion = CrossEntropyLoss()
criterion = torch.nn.NLLLoss()

# CV
results_wave = cross_validate(
    model, 
    dataset, 
    n_splits=5, 
    batch_size=1024, 
    num_epochs=1000, 
    lr=1e-3, 
    weight_decay=0., 
    device=device
)

In [9]:
from experiments.gcn import GCNClassifier
# Create your model, optimizer, and criterion
model = GCNClassifier(num_features=n_input, hidden_channels=16, num_classes=6)
optimizer = Adam(model.parameters(), lr=1e-3)
# criterion = CrossEntropyLoss()
criterion = torch.nn.NLLLoss()

# CV
results_gcn = cross_validate(
    model, 
    dataset, 
    n_splits=5, 
    batch_size=1024, 
    num_epochs=1000, 
    lr=1e-3, 
    weight_decay=0., 
    device=device
)

AttributeError: 'GCNClassifier' object has no attribute 'reset_parameters'

In [10]:
results_heat

,fold,train_loss,train_accuracy,validation_loss,validation_accuracy,training_time
0,0,1.115139,0.606250,1.596233,0.400000,76.438650
1,1,1.455811,0.416667,1.474891,0.416667,74.177841
2,2,1.423583,0.439583,1.567310,0.425000,74.710507
3,3,1.379605,0.485417,1.578432,0.450000,75.081440
4,4,1.364535,0.464583,1.546468,0.400000,76.879763


In [11]:
results_wave

,fold,train_loss,train_accuracy,validation_loss,validation_accuracy,training_time
0,0,1.051520,0.593750,1.615675,0.391667,137.646448
1,1,1.249877,0.522917,1.410082,0.500000,129.069187
2,2,1.351893,0.468750,4.014776,0.358333,129.899353
3,3,1.417571,0.427083,1.561825,0.433333,130.069394
4,4,1.526214,0.406250,1.670260,0.425000,130.660377


In [12]:
results_heat.validation_accuracy.mean()

0.41833333333333333

In [13]:
results_heat.validation_accuracy.std()

0.02074983266331455

In [14]:
results_wave.validation_accuracy.mean()

0.4216666666666667

In [15]:
results_wave.validation_accuracy.std()

0.052901900827012925